In [1]:
import pandas as pd
import datetime
import os
import matplotlib.pyplot as plt
import requests
import numpy as np
import folium


In [2]:
def get_osm_map_image(latitude, longitude, zoom=15, width=600, height=400):
    base_url = "https://www.openstreetmap.org/export/embed"
    
    # Set parameters for the API request
    params = {
        'bbox': f'{longitude-0.005},{latitude-0.005},{longitude+0.005},{latitude+0.005}',
        'layer': 'mapnik',
        'marker': f'{latitude},{longitude}',
        'zoom': zoom,
        'width': width,
        'height': height
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        # Save the map image to a file
        with open("osm_map_image.html", "w") as file:
            file.write(response.text)
        
        
    else:
        print(f"Failed to retrieve map image. Status code: {response.status_code}")


###### TEST to get new columns
def get_cols(df):
    grouped = df.groupby('Transformer')
    for i, group in grouped:
        
        group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
        group['Min duration of feeder overcurrent alarms (minutes)'] = group[['Min duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)']].min(axis=1)
        group['Max duration of feeder overcurrent alarms (minutes)'] = group[['Max duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L3 (min)']].max(axis=1)
        #group['Min duration of feeder overcurrent alarms'] = np.min(group['Min duration of overcurrent alarms L1 (min)'],group['Min duration of overcurrent alarms L2 (min)'],group['Min duration of overcurrent alarms L3 (min)'])
        #group['Max duration of feeder overcurrent alarms'] = np.max(group['Max duration of overcurrent alarms L1 (min)'],group['Max duration of overcurrent alarms L2 (min)'],group['Max duration of overcurrent alarms L3 (min)'])
        
        group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
        group['Min duration of feeder overcurrent alarms (minutes)'] = group['Min duration of feeder overcurrent alarms (minutes)'].min()
        group['Max duration of feeder overcurrent alarms (minutes)'] = group['Max duration of feeder overcurrent alarms (minutes)'].max()

        group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
        group['Min duration of power alarms (minutes)'] = group[['Min duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Min duration of power alarms L3 (min)']].min(axis=1)
        group['Max duration of power alarms (minutes)'] = group[['Max duration of power alarms L1 (min)','Max duration of power alarms L2 (min)','Max duration of power alarms L3 (min)']].max(axis=1)
        
        #group['Min duration of power alarms'] = np.min(group['Min duration of power alarms L1 (min)'],group['Min duration of power alarms L2 (min)'],group['Min duration of power alarms L3 (min)'])
        #group['Max duration of power alarms'] = np.max(group['Max duration of power alarms L1 (min)'],group['Max duration of power alarms L2 (min)'],group['Max duration of power alarms L3 (min)'])

        group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','Min duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)','Max duration of overcurrent alarms L3 (min)'],axis=1)
        group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','Min duration of power alarms L1 (min)','Max duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Max duration of power alarms L2 (min)','Min duration of power alarms L3 (min)','Max duration of power alarms L3 (min)'],axis=1)
        group = pd.DataFrame(group.iloc[0]).transpose()
        columns = group.columns 
        print(columns)
        return columns
    
def get_marker_color(value):
    if value > 0:
        return 'red'
    else:
        return 'blue'
    
def createMap(summary,summaryCols,month,year):
    m = folium.Map(location=[37.98392635351863, 23.727349678741202], tiles="OpenStreetMap", zoom_start=12)

    # Add points to the map
    for index, row in summary.iterrows():
        
        # tmp = pd.DataFrame(row[['Total energy consumption (KWh)','Nr. of Voltage unbalance alarms','% of time of Voltage unbalance alarms']])#.transpose()
        tmp = pd.DataFrame(row[summaryCols]).transpose()
        # tmp = tmp.reset_index(drop=True)
        # col = tmp.columns[0]
        newcol = row['Installation']+':'+row['Transformer']
        # tmp.rename(columns = {col:newcol},inplace=True)
        
        tmp = tmp.rename(index={tmp.index[0]: newcol})
        

        html = tmp.to_html(
        classes="table table-striped table-hover table-condensed table-responsive"
    )
        popup = folium.Popup(html)
        # popup = row['Installation']+'\n'+row['Transformer']+'\n'
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup, icon=folium.Icon(color=get_marker_color(row['Nr. of power alarms']))).add_to(m)

    m.save('HEDNO_KPIs_map_'+str(month)+'_'+str(year)+'.html')

In [3]:
year = 2024
for month in [1,2,3,4,5]:
    print(month)
    df = pd.read_excel('data/HEDNO_KPIs_'+str(month)+'_'+str(2024)+'.xlsx', engine='openpyxl')
    columns = get_cols(df)

    summary = pd.DataFrame(columns=columns)

    grouped = df.groupby('Transformer')
    for i, group in grouped:
        group['Nr. of feeder overcurrent alarms'] = group['Nr. of overcurrent alarms L1']+group['Nr. of overcurrent alarms L2']+group['Nr. of overcurrent alarms L3']
        group['Min duration of feeder overcurrent alarms (minutes)'] = group[['Min duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)']].min(axis=1)
        group['Max duration of feeder overcurrent alarms (minutes)'] = group[['Max duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L3 (min)']].max(axis=1)
        #group['Min duration of feeder overcurrent alarms'] = np.min(group['Min duration of overcurrent alarms L1 (min)'],group['Min duration of overcurrent alarms L2 (min)'],group['Min duration of overcurrent alarms L3 (min)'])
        #group['Max duration of feeder overcurrent alarms'] = np.max(group['Max duration of overcurrent alarms L1 (min)'],group['Max duration of overcurrent alarms L2 (min)'],group['Max duration of overcurrent alarms L3 (min)'])
        
        group['Nr. of feeder overcurrent alarms'] = group['Nr. of feeder overcurrent alarms'].sum()
        group['Min duration of feeder overcurrent alarms (minutes)'] = group['Min duration of feeder overcurrent alarms (minutes)'].min()
        group['Max duration of feeder overcurrent alarms (minutes)'] = group['Max duration of feeder overcurrent alarms (minutes)'].max()

        group['Nr. of power alarms'] = group['Nr. of power alarms L1']+group['Nr. of power alarms L2']+group['Nr. of power alarms L3']
        group['Min duration of power alarms (minutes)'] = group[['Min duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Min duration of power alarms L3 (min)']].min(axis=1)
        group['Max duration of power alarms (minutes)'] = group[['Max duration of power alarms L1 (min)','Max duration of power alarms L2 (min)','Max duration of power alarms L3 (min)']].max(axis=1)
        
        #group['Min duration of power alarms'] = np.min(group['Min duration of power alarms L1 (min)'],group['Min duration of power alarms L2 (min)'],group['Min duration of power alarms L3 (min)'])
        #group['Max duration of power alarms'] = np.max(group['Max duration of power alarms L1 (min)'],group['Max duration of power alarms L2 (min)'],group['Max duration of power alarms L3 (min)'])

        group = group.drop(['Nr. of overcurrent alarms L1','Nr. of overcurrent alarms L2','Nr. of overcurrent alarms L3','Min duration of overcurrent alarms L1 (min)','Max duration of overcurrent alarms L1 (min)','Min duration of overcurrent alarms L2 (min)','Max duration of overcurrent alarms L2 (min)','Min duration of overcurrent alarms L3 (min)','Max duration of overcurrent alarms L3 (min)'],axis=1)
        group = group.drop(['Nr. of power alarms L1','Nr. of power alarms L2','Nr. of power alarms L3','Min duration of power alarms L1 (min)','Max duration of power alarms L1 (min)','Min duration of power alarms L2 (min)','Max duration of power alarms L2 (min)','Min duration of power alarms L3 (min)','Max duration of power alarms L3 (min)'],axis=1)
        group = pd.DataFrame(group.iloc[0]).transpose()

        summary = summary.append(group, ignore_index=True)
    summary.loc[summary['Avg time of Voltage dips (msec)']<0,'Avg time of Voltage dips (msec)'] = 'Unknown'
    summaryCols = summary.drop(['Installation','Transformer','Distribution','Latitude','Longitude'],axis=1).columns
    summary.to_excel('data/HEDNO_KPIs_simplified_'+str(month)+'_'+str(year)+'.xlsx', index=False)
    summary = summary.fillna(0)
    
    createMap(summary,summaryCols,month,year)


1
Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ign

2
Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ign

3
Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ign

4
Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ign

5
Index(['Installation', 'Transformer', 'Distribution',
       'Total energy consumption (MWh)', 'Nr. of Voltage unbalance alarms',
       '% of time of Voltage unbalance alarms',
       'Nr. of Current unbalance alarms',
       '% of time of Current unbalance alarms', 'Nr. of Power Fails (outage)',
       'Min time of Power Fails (sec)', 'Max time of Power Fails (sec)',
       'Nr. of Voltage dips', 'Avg time of Voltage dips (msec)',
       'Nr. of Voltage swells', 'Avg time of Voltage swells (msec)',
       'Latitude', 'Longitude', 'Nr. of feeder overcurrent alarms',
       'Min duration of feeder overcurrent alarms (minutes)',
       'Max duration of feeder overcurrent alarms (minutes)',
       'Nr. of power alarms', 'Min duration of power alarms (minutes)',
       'Max duration of power alarms (minutes)'],
      dtype='object')


C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ignore_index=True)
C:\Users\EmilyP\AppData\Local\Temp\ipykernel_13268\654284991.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary = summary.append(group, ign